In [1]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
from torchvision import datasets, models

# Path to your data directory
data_dir = "/kaggle/input/plantvillage-dataset/color"

# Load the dataset without applying transformations initially
full_dataset = datasets.ImageFolder(root=data_dir)

# Split the dataset into training and temporary (validation + test)
train_indices, temp_indices = train_test_split(
    range(len(full_dataset)), test_size=0.2, random_state=42, stratify=full_dataset.targets
)

# Further split the temporary dataset into validation and test
val_indices, test_indices = train_test_split(
    temp_indices, test_size=0.5, random_state=42, stratify=[full_dataset.targets[i] for i in temp_indices]
)

# Subset the dataset using the indices
train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
val_dataset = torch.utils.data.Subset(full_dataset, val_indices)
test_dataset = torch.utils.data.Subset(full_dataset, test_indices)

# Define data transformations with data augmentation for the training set
train_dataset.dataset.transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224)),  # Randomly crop and resize to 224x224
    transforms.RandomHorizontalFlip(p=0.5),   # Randomly flip images horizontally
    transforms.RandomRotation(degrees=15),    # Randomly rotate images by up to 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random changes in brightness, contrast, etc.
    transforms.ToTensor(),                    # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for EfficientNetV2B0
])

# Define data transformations for the datasets
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to validation and test datasets
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

# Create DataLoader for each dataset
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load EfficientNetV2B0 pre-trained model
model = models.efficientnet_v2_s(pretrained=True)

# Modify the model's classifier for our problem (assuming we have N classes)
num_classes = len(full_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to evaluate the model on a given dataset (train, val, or test)
def evaluate(model, dataloader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient tracking during evaluation
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # Evaluate on the training and validation datasets
    train_accuracy = evaluate(model, train_loader)
    val_accuracy = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

# Final evaluation on the test dataset
test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")

torch.save(model.state_dict(), "efficientnet_v2_b0_trained_model.pth")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 98.1MB/s]


Epoch 1/10, Loss: 0.2114, Train Accuracy: 94.69%, Validation Accuracy: 93.54%
Epoch 2/10, Loss: 0.0562, Train Accuracy: 98.80%, Validation Accuracy: 97.92%
Epoch 3/10, Loss: 0.0484, Train Accuracy: 99.20%, Validation Accuracy: 98.69%
Epoch 4/10, Loss: 0.0323, Train Accuracy: 98.15%, Validation Accuracy: 97.51%
Epoch 5/10, Loss: 0.0395, Train Accuracy: 99.68%, Validation Accuracy: 99.15%
Epoch 6/10, Loss: 0.0242, Train Accuracy: 98.52%, Validation Accuracy: 97.94%
Epoch 7/10, Loss: 0.0245, Train Accuracy: 99.45%, Validation Accuracy: 98.78%
Epoch 8/10, Loss: 0.0263, Train Accuracy: 99.52%, Validation Accuracy: 98.77%
Epoch 9/10, Loss: 0.0259, Train Accuracy: 99.56%, Validation Accuracy: 99.01%
Epoch 10/10, Loss: 0.0210, Train Accuracy: 99.32%, Validation Accuracy: 98.60%
Test Accuracy: 98.71%
